In [3]:
from google.colab import files
uploaded = files.upload()

Saving restaurant_reviews_az.csv to restaurant_reviews_az.csv


In [4]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
import io
file_name = list(uploaded.keys())[0]  # Gets the uploaded file name dynamically
df = pd.read_csv(io.BytesIO(uploaded[file_name]))
display(df.head())

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",1/27/2020 22:59,1
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,4/19/2020 5:33,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2/29/2020 19:43,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,3/14/2020 21:47,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",1/17/2020 20:32,1


In [5]:
positive_reviews = df[df['Sentiment'] == 1].sample(n=50, random_state=42)
negative_reviews = df[df['Sentiment'] == 0].sample(n=50, random_state=42)
balanced_df = pd.concat([positive_reviews, negative_reviews]).reset_index(drop=True)
reviews = balanced_df['text'].tolist()
true_labels = balanced_df['Sentiment'].tolist()

display(balanced_df.head())


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,nAN_rYmPh82T1WzlROCcsw,8XeTv8Js_8um5Ht1Qnb0qw,n9kqlp48MzXB--LKoRjQhA,5,1,0,0,"First time ordering online, easy transaction, ...",1/4/2020 21:54,1
1,B0hgC22SWvPBStXv8jzSmw,8IPSQT6yPWmqxafauO4LrA,isrmmF6K_OZC2maNStwYNQ,5,0,0,0,This might be my favorite restaurant in Tucson...,7/18/2020 2:32,1
2,8j3H4k2gthWI3-AuqgqWzw,J7qboaD38ra2I0EMb3dqHA,eN-Zrz1orLoqIb7D6mUMbg,4,1,1,0,Ordered at the window! Because I ordered the b...,8/5/2020 21:52,1
3,KDtcFDryEmyJ0xZG_yy4rQ,vEFMvU78DrLZVKV1h-ZOpg,RSrBPqSze2HJkx5DZsm7FA,5,2,0,1,"Super good food, most of the menu is made from...",10/19/2020 4:16,1
4,I09Lr_K3DofaIHSiylVTFQ,DgtXZzWytUNWKyjq9i4dhw,l7FBm3yxW0dx0WqQVlcQ1Q,5,0,0,0,This place has amazing wings! The mild and buf...,1/10/2020 1:57,1


In [6]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def zero_shot_sentiment_analysis(reviews):
    predictions = []
    for review in reviews:
        result = sentiment_pipeline(review)[0]
        sentiment = result['label'].lower()
        if "positive" in sentiment:
            predictions.append(1)
        elif "negative" in sentiment:
            predictions.append(0)
        else:
            predictions.append(None)  # Handle unexpected responses
    return predictions

predicted_labels = zero_shot_sentiment_analysis(reviews)

valid_indices = [i for i, x in enumerate(predicted_labels) if x is not None]
filtered_true_labels = [true_labels[i] for i in valid_indices]
filtered_predicted_labels = [predicted_labels[i] for i in valid_indices]

accuracy = accuracy_score(filtered_true_labels, filtered_predicted_labels)
precision = precision_score(filtered_true_labels, filtered_predicted_labels)
recall = recall_score(filtered_true_labels, filtered_predicted_labels)
f1 = f1_score(filtered_true_labels, filtered_predicted_labels)

print("Zero-Shot Sentiment Analysis Results:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Zero-Shot Sentiment Analysis Results:
Accuracy: 0.86
Precision: 0.82
Recall: 0.92
F1 Score: 0.87


In [7]:
few_shot_examples = random.sample(reviews, 4)

def few_shot_sentiment_analysis(reviews, examples):
    predictions = []
    for review in reviews:
        # Construct prompt including few-shot examples
        prompt = "Here are some examples of sentiment analysis:\n"
        for ex in examples:
            sentiment_label = "positive" if true_labels[reviews.index(ex)] == 1 else "negative"
            prompt += f"Review: {ex}\nSentiment: {sentiment_label}\n"
        prompt += f"Now classify the following:\nReview: {review}\nSentiment: "

        result = sentiment_pipeline(review)[0]
        sentiment = result['label'].lower()
        if "positive" in sentiment:
            predictions.append(1)
        elif "negative" in sentiment:
            predictions.append(0)
        else:
            predictions.append(None)  # Handle unexpected responses
    return predictions

few_shot_predicted_labels = few_shot_sentiment_analysis(reviews, few_shot_examples)

accuracy_fs = accuracy_score(true_labels, few_shot_predicted_labels)
precision_fs = precision_score(true_labels, few_shot_predicted_labels)
recall_fs = recall_score(true_labels, few_shot_predicted_labels)
f1_fs = f1_score(true_labels, few_shot_predicted_labels)

print("Few-Shot Sentiment Analysis Results:")
print(f"Accuracy: {accuracy_fs:.2f}")
print(f"Precision: {precision_fs:.2f}")
print(f"Recall: {recall_fs:.2f}")
print(f"F1 Score: {f1_fs:.2f}")


Few-Shot Sentiment Analysis Results:
Accuracy: 0.86
Precision: 0.82
Recall: 0.92
F1 Score: 0.87


In [8]:
llama_pipeline = pipeline("text-classification", model="facebook/bart-large-mnli")

def multi_llm_sentiment_analysis(reviews):
    results = {"distilbert": [], "llama": []}
    for review in reviews:
        distilbert_result = sentiment_pipeline(review)[0]['label'].lower()
        llama_result = llama_pipeline(review)[0]['label'].lower()

        results["distilbert"].append(1 if "positive" in distilbert_result else 0)
        results["llama"].append(1 if "positive" in llama_result else 0)
    return results

multi_llm_results = multi_llm_sentiment_analysis(reviews)

print("Comparison of Multiple LLMs")
print("DistilBERT Sentiment Predictions:", multi_llm_results["distilbert"][:5])
print("LLaMA Sentiment Predictions:", multi_llm_results["llama"][:5])


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Comparison of Multiple LLMs
DistilBERT Sentiment Predictions: [1, 0, 1, 1, 1]
LLaMA Sentiment Predictions: [0, 0, 0, 0, 0]
